# End to End Example of Using the `aisquared` Python SDK to Create a Dashboard Integration Using a `ReverseMLWorkflow`

In this tutorial, we will go over the creation of a `.air` file that uses a `ReverseMLWorkflow` to pull data from an Amazon S3 bucket and populate a dashboard into multiple end user applications.

This notebook differs only slightly from the `End-to-end Dashboard.ipynb` example also in this repository.  The main difference between the two notebooks is the `query_selector` parameter in the rendering steps.  The other notebook only has the first selector published, meaning it only looks for the "general" tab in Dynamics 365. This notebook expands which selection and injection criteria are expected, allowing the resulting `.air` file to inject on multiple pages.

# Importing Packages

Because the data to be used for the Reverse ML Workflow is already stored in S3, we only need to import the `config` subpackage from the `aisquared` package to complete this tutorial.

In [1]:
# import the aisquared package
from aisquared import config

from aisquared.base import CustomObject

# Defining the Workflow

Now that we have imported the required package, let's dive head first into completing this workflow.  We are going to need to perform the following steps:

- Create a harvester which looks for customer names in the webpage
- Connect the webpage and the text we have harvested to the S3 bucket for query using the `ReverseMLWorkflow`
- Render results in a set of steps to create the dashboard using the data related to the customer

In [2]:
# Create the harvester
harvester = config.harvesting.TextHarvester(
    how = 'regex',
    regex = 'Alice Alison|Bob Bobson|Charles Charleston|David Davidson|Eric Ercison|Frederick Frederickson|Georg Georges|Hilda Henrick|Igor Igloo|John Johnson',
    limit = 1
)

# No preprocesser is required for this dashboard
preprocesser = None

# The analytic in this workflow is a ReverseMLWorkflow
analytic = config.analytic.ReverseMLWorkflow(
    column = 'Lead_Name',
    period = 1,
    filename = 'SOS_Sample_for_Dashboard_edit9.csv',
    input_type = 'text',
    bucket = 'ai2-demo-bucket'
)

# No postprocesser is required for this dashboard
postprocesser = None

# Rendering steps for the dashboard
rendering_steps = [

    # Create the global container that will encompass the entire dashboard
    config.rendering.ContainerRendering(
        label = 'Dashboard Container',
        id = 'ai2-sos-dashboard',
        query_selector = "[data-id='tabpanel-general'], [data-id='tabpanel-SUMMARY_TAB'], [class='record-page-decorator'], [data-test='fullscreen-layout']",
        height = '75'
    ),
    
    # Create the book of business rendering class
    CustomObject(
        class_name = 'BookOfBusinessRendering',
        containerId = 'ai2-sos-dashboard'
    )
]

# Compile Into a `.air` File

Now that we have all of the steps, all we have to do is put them together and compile them into a `.air` file to be uploaded into the platform!

In [3]:
# Compile all of the configuration information into a ModelConfiguration class, to
# be compiled into a .air file
model_config = config.ModelConfiguration(
    'Python SDK Book Of Business Dashboard',
    harvesting_steps=harvester,
    preprocessing_steps=preprocesser,
    analytic=analytic,
    postprocessing_steps=postprocesser,
    rendering_steps=rendering_steps,
    description='Version of the SOS Dashboard .air file created using the Python SDK'
)

# Compile the model
model_config.compile()


In [4]:
!ls

Book of Business Rendering.ipynb
CIFAR10 Classifier.air
End-to-end Dashboard Multiple Apps With Doughnut Chart.ipynb
End-to-end Dashboard Multiple Apps.ipynb
End-to-end Dashboard.ipynb
Image Classification Model.ipynb
LICENSE
LocalAnalytic.air
MyFirstAIRFile.air
Python SDK Book Of Business Dashboard.air
Python SDK SOS Dashboard Dynamics 365.air
Python SDK SOS Dashboard Multiple Applications With Doughnut Chart.air
Python SDK SOS Dashboard Multiple Applications.air
README.md
Simple Local Analytic.ipynb
aisquared Introduction.ipynb
analytic.json
cifar10_classifier.h5
config.json
simple_analytic.json


# Upload the file to the platform

We are done!  Now, you should have a file called `Python SDK SOS Dashboard Multiple Applications` in this directory.  Go ahead and upload the file to the platform to start seeing results!